In [3]:
import pandas as pd

# Tu DataFrame original
data = {
    'sequence id': ['A0A1Y2ANA8.1/421-668', 'A0A1Y2ANA8.1/421-668', 'A0A1Y2ANA8.1/421-668',
                    'A0A5J5ICB5.1/11-99', 'A0A5J5ICB5.1/11-99', 'A0A5J5ICB5.1/11-99'],
    'residue number': [1, 2, 3, 1, 2, 3],
    'conservation score': [0.105774, 0.202515, 0.141357, 0.531250, 0.302979, 0.243530]
}

df = pd.DataFrame(data)

# Convertir 'conservation score' a cadena antes de unir
df['conservation score'] = df['conservation score'].astype(str)

# Agrupar y concatenar los valores de 'conservation score'
result_df = df.groupby('sequence id')['conservation score'].agg(' '.join).reset_index()

print(result_df)


            sequence id          conservation score
0  A0A1Y2ANA8.1/421-668  0.105774 0.202515 0.141357
1    A0A5J5ICB5.1/11-99    0.53125 0.302979 0.24353


In [4]:
import pandas as pd
from scipy.stats import pearsonr

# Lee los resultados de conservación predichos
predicted_df = pd.read_csv("output_results_curated_8M.csv")

# Lee las primeras 100 puntuaciones de conservación reales del archivo conservation_scores.csv
true_df = pd.read_csv("curated_dataset/conservation_scores.csv", delimiter='\t')
true_df = true_df.groupby('sequence id').head(1).head(100).reset_index(drop=True)

# Extrae la parte común de las secuencias (puedes ajustar esto según tus necesidades)
predicted_df['common_sequence'] = predicted_df['sequence'].str.extract(r'([A-Za-z0-9]+\.[0-9]+/[0-9]+-[0-9]+)')
true_df['common_sequence'] = true_df['sequence id'].str.extract(r'([A-Za-z0-9]+\.[0-9]+/[0-9]+-[0-9]+)')

# Fusionamos los DataFrames por la columna 'common_sequence'
combined_df = pd.merge(predicted_df, true_df, how='inner',
                       left_on='common_sequence', right_on='common_sequence')

# Convertimos los valores de 'conservation' y 'conservation score' a listas de números
combined_df['conservation'] = combined_df['conservation'].apply(lambda x: [float(val) for val in x.split()])
combined_df['conservation score'] = combined_df['conservation score'].apply(lambda x: [float(val) for val in x.split()])

# Calculamos el coeficiente de Pearson por residuo
pearson_coefficient, _ = pearsonr(
    combined_df['conservation'].explode(), combined_df['conservation score'].explode())
print(f"Coeficiente de Pearson por residuo (primeras 100 filas): {pearson_coefficient:.4f}")


KeyError: 'sequence id'